# Python code to replicate the illustrative example from "Double Robust Variance Estimation"

Code used to replicate the analysis for the effect of maternal anemia on birth weight among women living with HIV was estimated with data from the Improving Pregnancy Outcomes with Progesterone (IPOP) trial described in Shook-Sa BE, Zivich PN, Lee C, Xue K, Ross RK, Edwards JK, Stringer JSA, Cole SR. "Double Robust Variance Estimation" Submitted 2024.

Paul Zivich (2024/11/01)

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import formulaic as formula
from formulaic import model_matrix
import delicatessen as deli
from delicatessen import MEstimator
from delicatessen.estimating_equations import ee_regression

from efuncs import ee_aipw_plugin, ee_aipw_wreg, ee_tmle, bound_unit
from var_helper import inf_func_inference, aipw_plugin_bootstrap, aipw_wreg_bootstrap, tmle_bootstrap

np.random.seed(7777777)

print("Versions")
print("====================")
print("NumPy:       ", np.__version__)
print("SciPy:       ", sp.__version__)
print("Pandas:      ", pd.__version__)
print("formulaic:   ", formula.__version__)
print("Delicatessen:", deli.__version__)

Versions
NumPy:        1.25.2
SciPy:        1.11.2
Pandas:       1.4.1
formulaic:    0.5.2
Delicatessen: 2.4


## Loading data and setup

In [2]:
d = pd.read_csv("data/ipop_subset_04.15.24.csv")
d['intercept'] = 1

# Applying intervention to data
d1 = d.copy()         # Copy data
d1['anemia'] = 1      # And set anemia equal to 1
d0 = d.copy()         # Copy data
d0['anemia'] = 0      # And set anemia equal to 0

# Extracting outcome and action variables into arrays
a = np.asarray(d['anemia'])
y = np.asarray(d['del_bw'])

# Empty list for results storage
rows = []

## Setting up estimating equations

In [3]:
def psi_aipw_plugin(theta):
    # Estimating function for the plug-in AIPW with correct models
    return ee_aipw_plugin(theta=theta,         # Parameter vector
                          y=y,                 # Outcome column
                          a=a,                 # Action column
                          PSM=ps_dmatrix,      # Propensity score model
                          OM=out_dmatrix,      # Outcome model
                          OM1=out1_dmatrix,    # Outcome model but with anemia=1
                          OM0=out0_dmatrix)    # Outcome model but with anemia=0


def psi_aipw_wreg(theta):
    # Estimating function for the plug-in AIPW with correct models
    return ee_aipw_wreg(theta=theta,       # Parameter vector
                        y=y,               # Outcome column
                        a=a,               # Action column
                        PSM=ps_dmatrix,    # Propensity score model
                        OM=out_dmatrix,    # Outcome model
                        OM1=out1_dmatrix,  # Outcome model but with anemia=1
                        OM0=out0_dmatrix)  # Outcome model but with anemia=0


def psi_tmle(theta):
    return ee_tmle(theta=theta,       # Parameter vector
                   y=y,               # Outcome column
                   a=a,               # Action column
                   PSM=ps_dmatrix,    # Propensity score model
                   OM=out_dmatrix,    # Outcome model
                   OM1=out1_dmatrix,  # Outcome model but with anemia=1
                   OM0=out0_dmatrix)  # Outcome model but with anemia=0


def psi_bounded_outcome(theta):
    continuous_bound = 1e-5
    min_y = np.min(y) - continuous_bound
    max_y = np.max(y) + continuous_bound
    yb = bound_unit(y, mini=min_y, maxi=max_y)
    return ee_regression(theta=theta, X=out_dmatrix, y=yb, model='linear')

## Full Covariate Specification

The following models use the full covariate set specification, which correspond to `ps_model_full` and `outcome_model_full`.

In [4]:
rows.append(["Full-Covariate", ])      # Label for the table output

In [5]:
# Propensity score model specifications
psmodel = ("base_art + base_alc + first_trimester + C(age_cat) + C(num_prior_births) "
           "+ Height_SP1 + Height_SP2 + Height_SP3 + Height_SP4")
ps_dmatrix = model_matrix(psmodel, d)

# Outcome model specifications
ymodel = ("anemia + base_art + base_alc + first_trimester + C(age_cat) + C(num_prior_births) + "
          "Height_SP1 + Height_SP2 + Height_SP3 + Height_SP4")
out_dmatrix = model_matrix(ymodel, d)
out1_dmatrix = model_matrix(ymodel, d1)
out0_dmatrix = model_matrix(ymodel, d0)

### Plug-in AIPW

In [6]:
# Initial values (generic but causal means near observed mean)
init_vals = [0., 3000., 3000., ] + [0., ]*ps_dmatrix.shape[1] + [3000., ] + [0., ]*(out_dmatrix.shape[1] - 1)

# Applying M-estimator
estr = MEstimator(psi_aipw_plugin, init=init_vals)
estr.estimate(maxiter=20000)
ci = estr.confidence_intervals()

# Propensity score model solutions to simplify other estimators root-finding
ps_solutions = estr.theta[3: 3 + ps_dmatrix.shape[1]]

# Computing the influence-function variance by hand
ifvar = inf_func_inference(theta=estr.theta, y=y, a=a,
                           PSM=ps_dmatrix, OM=out_dmatrix,
                           OM1=out1_dmatrix, OM0=out0_dmatrix)

# Bootstrapping procedure
bsvar = aipw_plugin_bootstrap(data=d, y='del_bw', a='anemia', a_model=psmodel, y_model=ymodel, 
                              iterations=5000)

# Storing results for the output
rows.append(["Plug-in", estr.theta[0], estr.variance[0, 0]**0.5, ci[0, 0], ci[0, 1],
             ifvar[0], ifvar[1], ifvar[2],
             bsvar[0], bsvar[1], bsvar[2]])

### Weighted-regression AIPW

In [7]:
# Weighted-regression AIPW estimator

def psi_aipw_wreg_correct(theta):
    # Estimating function for the weighted-regression AIPW with correct models
    return ee_aipw_wreg(theta=theta,            # Parameter vector
                        y=y,                    # Outcome column
                        a=a,                    # Action column
                        PSM=ps_model_full,      # Propensity score model
                        OM=out_model_full,      # Outcome model
                        OM1=out1_model_full,    # Outcome model but with anemia=1
                        OM0=out0_model_full)    # Outcome model but with anemia=0


# Initial values (uses previous PS solution to speed up)
init_vals = [0., 3000., 3000., ] + list(ps_solutions) + [3000., ] + [0., ]*(out_dmatrix.shape[1] - 1)

# Applying M-estimator
estr = MEstimator(psi_aipw_wreg, init=init_vals)
estr.estimate(maxiter=20000)
ci = estr.confidence_intervals()

# Computing the influence-function variance by hand
ifvar = inf_func_inference(theta=estr.theta, y=y, a=a, PSM=ps_dmatrix, OM=out_dmatrix,
                           OM1=out1_dmatrix, OM0=out0_dmatrix)

# Bootstrapping procedure
bsvar = aipw_wreg_bootstrap(data=d, y='del_bw', a='anemia', a_model=psmodel, y_model=ymodel, 
                            iterations=5000)

# Storing results for the output
rows.append(["WReg", estr.theta[0], estr.variance[0, 0]**0.5, ci[0, 0], ci[0, 1],
             ifvar[0], ifvar[1], ifvar[2],
             bsvar[0], bsvar[1], bsvar[2]])

### TMLE

In [8]:
# Initial values (uses previous PS solution to speed up)
init_vals = [0., 3000., 3000.] + list(ps_solutions) + [0., ]*out_dmatrix.shape[1] + [0., 0.]

# Applying M-estimator
estr = MEstimator(psi_tmle, init=init_vals)
estr.estimate(maxiter=20000)
ci = estr.confidence_intervals()

# Computing the influence-function variance by hand
ifvar = inf_func_inference(theta=estr.theta, y=y, a=a, PSM=ps_dmatrix, OM=out_dmatrix,
                           OM1=out1_dmatrix, OM0=out0_dmatrix, unbound_y=True)

# Bootstrapping procedure
bsvar = tmle_bootstrap(data=d, y='del_bw', a='anemia', a_model=psmodel, y_model=ymodel, 
                       iterations=5000)

# Storing results for the output
rows.append(["TMLE", estr.theta[0], estr.variance[0, 0]**0.5, ci[0, 0], ci[0, 1],
             ifvar[0], ifvar[1], ifvar[2],
             bsvar[0], bsvar[1], bsvar[2]])

c:\users\zivic\documents\open-source\delicatessen\delicatessen\utilities.py:25: RuntimeWarning: divide by zero encountered in log
  return np.log(prob / (1 - prob))


## Naive Outcome Specifications

In [9]:
rows.append(["Naive Outcome", ])       # Label for the table output

In [10]:
# Propensity score model specifications
psmodel = ("base_art + base_alc + first_trimester + C(age_cat) + C(num_prior_births) "
           "+ Height_SP1 + Height_SP2 + Height_SP3 + Height_SP4")
ps_dmatrix = model_matrix(psmodel, d)

# Outcome model specifications
ymodel = "anemia"
out_dmatrix = model_matrix(ymodel, d)
out1_dmatrix = model_matrix(ymodel, d1)
out0_dmatrix = model_matrix(ymodel, d0)

### Plug-in AIPW

In [11]:
# Initial values (generic but causal means near observed mean)
init_vals = [0., 3000., 3000., ] + [0., ]*ps_dmatrix.shape[1] + [3000., 0., ]

# Applying M-estimator
estr = MEstimator(psi_aipw_plugin, init=init_vals)
estr.estimate(maxiter=20000)
ci = estr.confidence_intervals()

# Computing the influence-function variance by hand
ifvar = inf_func_inference(theta=estr.theta, y=y, a=a, PSM=ps_dmatrix, OM=out_dmatrix,
                           OM1=out1_dmatrix, OM0=out0_dmatrix)

# Bootstrapping procedure
bsvar = aipw_plugin_bootstrap(data=d, y='del_bw', a='anemia', a_model=psmodel, y_model=ymodel, 
                              iterations=5000)

# Storing results for the output
rows.append(["Plug-in", estr.theta[0], estr.variance[0, 0]**0.5, ci[0, 0], ci[0, 1],
             ifvar[0], ifvar[1], ifvar[2],
             bsvar[0], bsvar[1], bsvar[2]])

### Weighted-regression AIPW

In [12]:
# Initial values (generic but causal means near observed mean)
init_vals = [0., 3000., 3000., ] + [0., ]*ps_dmatrix.shape[1] + [3000., 0.]

# Applying M-estimator
estr = MEstimator(psi_aipw_wreg, init=init_vals)
estr.estimate(maxiter=20000)
ci = estr.confidence_intervals()

# Computing the influence-function variance by hand
ifvar = inf_func_inference(theta=estr.theta, y=y, a=a, PSM=ps_dmatrix, OM=out_dmatrix,
                           OM1=out1_dmatrix, OM0=out0_dmatrix)

# Bootstrapping procedure
bsvar = aipw_wreg_bootstrap(data=d, y='del_bw', a='anemia', a_model=psmodel, y_model=ymodel, 
                            iterations=5000)

# Storing results for the output
rows.append(["WReg", estr.theta[0], estr.variance[0, 0]**0.5, ci[0, 0], ci[0, 1],
             ifvar[0], ifvar[1], ifvar[2],
             bsvar[0], bsvar[1], bsvar[2]])

### TMLE

In [13]:
# Initial values (uses previous PS solution to speed up)
init_vals = [0., 3000., 3000., ] + [0., ]*ps_dmatrix.shape[1] + [0., 0.] + [0., 0., ]

# Applying M-estimator
estr = MEstimator(psi_tmle, init=init_vals)
estr.estimate(maxiter=20000)
ci = estr.confidence_intervals()

# Computing the influence-function variance by hand
ifvar = inf_func_inference(theta=estr.theta, y=y, a=a, PSM=ps_dmatrix, OM=out_dmatrix,
                           OM1=out1_dmatrix, OM0=out0_dmatrix, unbound_y=True)

# Bootstrapping procedure
bsvar = tmle_bootstrap(data=d, y='del_bw', a='anemia', a_model=psmodel, y_model=ymodel, iterations=5000)

# Storing results for the output
rows.append(["TMLE", estr.theta[0], estr.variance[0, 0]**0.5, ci[0, 0], ci[0, 1],
             ifvar[0], ifvar[1], ifvar[2],
             bsvar[0], bsvar[1], bsvar[2]])

c:\users\zivic\documents\open-source\delicatessen\delicatessen\utilities.py:25: RuntimeWarning: divide by zero encountered in log
  return np.log(prob / (1 - prob))
c:\users\zivic\documents\open-source\delicatessen\delicatessen\utilities.py:25: RuntimeWarning: invalid value encountered in log
  return np.log(prob / (1 - prob))


## Naive Propensity Score Specifications

In [14]:
rows.append(["Naive PS", ])            # Label for the table output

In [15]:
# Propensity score model specifications
psmodel = "1"
ps_dmatrix = model_matrix(psmodel, d)

# Outcome model specifications
ymodel = ("anemia + base_art + base_alc + first_trimester + C(age_cat) + C(num_prior_births) + "
          "Height_SP1 + Height_SP2 + Height_SP3 + Height_SP4")
out_dmatrix = model_matrix(ymodel, d)
out1_dmatrix = model_matrix(ymodel, d1)
out0_dmatrix = model_matrix(ymodel, d0)

### Plug-in AIPW

In [16]:
# Initial values (generic but causal means near observed mean)
init_vals = [0., 3000., 3000., ] + [0., ] + [3000., ] + [0., ]*(out_dmatrix.shape[1] - 1)

# Applying M-estimator
estr = MEstimator(psi_aipw_plugin, init=init_vals)
estr.estimate(maxiter=20000)
ci = estr.confidence_intervals()

# Computing the influence-function variance by hand
ifvar = inf_func_inference(theta=estr.theta, y=y, a=a, PSM=ps_dmatrix, OM=out_dmatrix,
                           OM1=out1_dmatrix, OM0=out0_dmatrix)

# Bootstrapping procedure
bsvar = aipw_plugin_bootstrap(data=d, y='del_bw', a='anemia', a_model=psmodel, y_model=ymodel, iterations=5000)

# Storing results for the output
rows.append(["Plug-in", estr.theta[0], estr.variance[0, 0]**0.5, ci[0, 0], ci[0, 1],
             ifvar[0], ifvar[1], ifvar[2],
             bsvar[0], bsvar[1], bsvar[2]])

### Weighted-regression AIPW

In [17]:
# Initial values (generic but causal means near observed mean)
init_vals = [0., 3000., 3000., ] + [0., ] + [3000., ] + [0., ]*(out_dmatrix.shape[1] - 1)

# Applying M-estimator
estr = MEstimator(psi_aipw_wreg, init=init_vals)
estr.estimate(maxiter=20000)
ci = estr.confidence_intervals()

# Computing the influence-function variance by hand
ifvar = inf_func_inference(theta=estr.theta, y=y, a=a, PSM=ps_dmatrix, OM=out_dmatrix,
                           OM1=out1_dmatrix, OM0=out0_dmatrix)

# Bootstrapping procedure
bsvar = aipw_wreg_bootstrap(data=d, y='del_bw', a='anemia', a_model=psmodel, y_model=ymodel, iterations=5000)

# Storing results for the output
rows.append(["WReg", estr.theta[0], estr.variance[0, 0]**0.5, ci[0, 0], ci[0, 1],
             ifvar[0], ifvar[1], ifvar[2],
             bsvar[0], bsvar[1], bsvar[2]])

### TMLE

In [18]:
# Initial values (uses previous PS solution to speed up)
init_vals = [0., ]*out_dmatrix.shape[1]
estr = MEstimator(psi_bounded_outcome, init=init_vals)
estr.estimate(maxiter=20000, deriv_method='exact')
out_solution = list(estr.theta)
init_vals = [0., 3000., 3000., ] + [0., ] + out_solution + [0., 0., ]

# Applying M-estimator
estr = MEstimator(psi_tmle, init=init_vals)
estr.estimate(maxiter=20000)
ci = estr.confidence_intervals()

# Computing the influence-function variance by hand
ifvar = inf_func_inference(theta=estr.theta, y=y, a=a, PSM=ps_dmatrix, OM=out_dmatrix,
                           OM1=out1_dmatrix, OM0=out0_dmatrix, unbound_y=True)

# Bootstrapping procedure
bsvar = tmle_bootstrap(data=d, y='del_bw', a='anemia', a_model=psmodel, y_model=ymodel, iterations=5000)

# Storing results for the output
rows.append(["TMLE", estr.theta[0], estr.variance[0, 0]**0.5, ci[0, 0], ci[0, 1],
             ifvar[0], ifvar[1], ifvar[2],
             bsvar[0], bsvar[1], bsvar[2]])

## Displaying the Results

In [19]:
results = pd.DataFrame(rows, 
                       columns=["Estimator", "Est", 
                                "ES-SE", "ES-LCL", "ES-UCL", 
                                "IF-SE", "IF-LCL", "IF-UCL",
                                "BS-SE", "BS-LCL", "BS-UCL"])
results = results.set_index("Estimator")
pd.set_option('display.max_columns', None)
pd.set_option('expand_frame_repr', False)
results.round(0)

,Est,ES-SE,ES-LCL,ES-UCL,IF-SE,IF-LCL,IF-UCL,BS-SE,BS-LCL,BS-UCL
Estimator,,,,,,,,,,
Full-Covariate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Plug-in,-37.0,56.0,-147.0,73.0,58.0,-151.0,76.0,62.0,-159.0,84.0
WReg,-41.0,56.0,-151.0,69.0,56.0,-151.0,68.0,62.0,-162.0,79.0
TMLE,-37.0,56.0,-147.0,73.0,58.0,-151.0,76.0,62.0,-160.0,85.0
Naive Outcome,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Plug-in,-36.0,57.0,-148.0,76.0,61.0,-157.0,85.0,64.0,-162.0,90.0
WReg,-36.0,57.0,-148.0,77.0,61.0,-156.0,84.0,64.0,-162.0,90.0
TMLE,-36.0,57.0,-148.0,77.0,61.0,-156.0,84.0,62.0,-157.0,86.0
Naive PS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


END